In [4]:
from astropy.io import fits
from glob import glob
import os
import matplotlib as mpl
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from jwst import datamodels
from itertools import product
import itertools
from astropy.wcs import WCS
import math
from IPython.display import display
from visualization_helpers import *

In [5]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

# figure sizes and resolutions
rcParams['figure.figsize'] = [20,20]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

cmap = plt.get_cmap('summer')
cmaplist = [cmap(i) for i in range(cmap.N)]

# create the new map
cmap = cmap.from_list('Custom cmap', cmaplist[0:], cmap.N)

In [6]:
import jwst
print(jwst.__version__)

1.11.3


> All information that is provided below can be found in this <a href="https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/science_products.html">link</a>

# S

### RATEINTS and CALINTS

#### RATEINTS

Countrate products are produced by applying the ramp_fitting step to the integrations within an exposure, in order to compute count rates from the original accumulating signal ramps. For exposures that contain multiple integrations (NINTS > 1) this is done in two ways, which results in two separate products. First, countrates are computed for each integration within the exposure, the results of which are stored in a rateints product. These products contain 3-D data arrays, where each plane of the data cube contains the countrate image for a given integration. 

| HDU | EXTNAME | HDU Type | Data Type | Dimensions | 
| --- | --- | --- | --- | --- | 
| 0 | N/A | primary | N/A | N/A
| 1 | SCI | IMAGE | float32 | ncols x nrows x nints | 
| 2 | ERR | IMAGE | float32 | ncols x nrows x nints 
| 3 | DQ  | IMAGE | uint32 | ncols x nrows x nints | 
| 4 | INT_TIMES | BINTABLE | N/A | nints (rows) x 7 cols
| 5 | VAR_POISSON | IMAGE | float32 | ncols x nrows x nints | 
| 6 | VAR_RNOISE | IMAGE | float32 | N/A
| 7 | ASDF | BINTABLE | N/A | variable | 


* SCI: 3-D data array containing the pixel values, in units of DN/s. The first two dimensions are equal to the size of the detector readout, with the data from multiple integrations stored along the 3rd axis.

* ERR: 3-D data array containing uncertainty estimates on a per-integration basis. These values are based on the combined VAR_POISSON and VAR_RNOISE data (see below), given as standard deviation.

* DQ: 3-D data array containing DQ flags. Each plane of the cube corresponds to a given integration

#### CALINTS

Single exposure calibrated products duplicate a lot of the format and content of countrate products. There are two different high-level forms of calibrated products: one containing results for all integrations in an exposure (calints) and one for results averaged over all integrations (cal). These products are the main result of Stage 2 pipelines like calwebb_image2 and calwebb_spec2. There are many additional types of extensions that only appear for certain observing modes or instruments, especially for spectroscopic exposures.

| HDU | EXTNAME | HDU Type | Data Type | Dimensions | 
| --- | --- | --- | --- | --- | 
| 0 | N/A | primary | N/A | N/A
| 1 | SCI | IMAGE | float32 | ncols x nrows x nints | 
| 2 | ERR | IMAGE | float32 | ncols x nrows x nints |
| 3 | DQ  | IMAGE | uint32 | ncols x nrows x nints | 
|   | INT_TIMES | BINTABLE | N/A | nints (rows) x 7 cols |
|   | VAR_POISSON | IMAGE | float32 | ncols x nrows x nints | 
|   | VAR_RNOISE | IMAGE | float32 | N/A | 
|   | VAR_FLAT | IMAGE | float32 | ncols x nrows x nints | 
|   | AREA | IMAGE |  | ncols x nrows |
|   | WAVELENGTH* | IMAGE | float32 | ncols x nrows |
|   | ASDF | BINTABLE | N/A | variable | 

* SCI: 3-D data array containing the pixel values, in units of surface brightness, for each integration.

* ERR: 3-D data array containing uncertainty estimates for each pixel, for each integration. These values are based on the combined VAR_POISSON and VAR_RNOISE data (see below), given as standard deviation.

* DQ: 3-D data array containing DQ flags for each pixel, for each integration.

* INT_TIMES: A table of beginning, middle, and end time stamps for each integration in the exposure.

* VAR_POISSON: 3-D data array containing the variance estimate for each pixel, based on Poisson noise only, for each integration.

* VAR_RNOISE: 3-D data array containing the variance estimate for each pixel, based on read noise only, for each integration.

* VAR_FLAT: 2-D data array containing the variance estimate for each pixel, based on uncertainty in the flat-field.

* AREA: 2-D data array containing pixel area values, added by the photom step, for imaging modes.

* WAVELENGTH: 2-D data array of wavelength values for each pixel, for some spectroscopic modes.

* ADSF: The data model meta data.

### PSFSTACK, I2D, CRFINT, PSFSUB and PSFALIGN

#### PSFSTACK

The stack_refs step in the calwebb_coron3 pipeline takes a collection of PSF reference image and assembles them into a 3-D stack of PSF images, which results in a psfstack product. The psfstack product uses the CubeModel data model, which when serialized to a FITS file has the structure shown below.

| HDU | EXTNAME | HDU Type | Data Type | Dimensions | 
| --- | --- | --- | --- | --- | 
| 0 | N/A | primary | N/A | N/A
| 1 | SCI | IMAGE | float32 | ncols x nrows x nints | 
| 2 | DQ  | IMAGE | uint32 | ncols x nrows x nints | 
| 3 | ERR | IMAGE | float32 | ncols x nrows x nints |
| 4 | ASDF | BINTABLE | N/A | variable

* SCI: 3-D data array containing a stack of 2-D PSF images.

* DQ: 3-D data array containing DQ flags for each PSF image.

* ERR: 3-D data array containing a stack of 2-D uncertainty estimates for each PSF image.

* ADSF: The data model meta data.

#### I2D

Images that have been resampled by the resample step use a different set of data arrays than other science products. Resampled 2-D images are stored in <b>i2d</b> products.

| HDU | EXTNAME | HDU Type | Data Type | Dimensions | 
| --- | --- | --- | --- | --- | 
| 0 | N/A | primary | N/A | N/A
| 1 | SCI | IMAGE | float32 | ncols x nrows | 
| 2 | ERR | IMAGE | float32 | ncols x nrows |
| 3 | CON  | IMAGE | int32 | ncols x nrows x nplanes | 
| 4 | WHT  | IMAGE | float32 | ncols x nrows | 
| 5 | VAR_POISSON  | IMAGE | float32 | ncols x nrows | 
| 6 | VAR_RNOISE  | IMAGE | float32 | ncols x nrows | 
| 6 | VAR_FLAT  | IMAGE | float32 | ncols x nrows | 
|   | HDRTAB* | BINTABLE | N/A | variable
|   | ASDF | BINTABLE | N/A | variable

- SCI: 2-D data array containing the pixel values, in units of surface brightness

- ERR: 2-D data array containing resampled uncertainty estimates, given as standard deviation

- CON: 3-D context image, which encodes information about which input images contribute to a specific output pixel

- WHT: 2-D weight image giving the relative weight of the output pixels

- VAR_POISSON: 2-D resampled Poisson variance estimates for each pixel

- VAR_RNOISE: 2-D resampled read noise variance estimates for each pixel

- VAR_FLAT: 2-D resampled flat-field variance estimates for each pixel

- HDRTAB: A table containing meta data (FITS keyword values) for all of the input images that were combined to produce the output image. Only appears when multiple inputs are used.

- ADSF: The data model meta data.

#### PSFALIGN

The align_refs step in the calwebb_coron3 pipeline creates a 3-D stack of PSF images that are aligned to corresponding science target images. The resulting psfalign product uses the QuadModel data model, which when serialized to a FITS file has the structure and content shown below.

| HDU | EXTNAME | HDU Type | Data Type | Dimensions | 
| --- | --- | --- | --- | --- | 
| 0 | N/A | primary | N/A | N/A
| 1 | SCI | IMAGE | float32 | ncols x nrows x npsfs x nints | 
| 2 | DQ  | IMAGE | uint32 | ncols x nrows x npsfs xnints | 
| 3 | ERR | IMAGE | float32 | ncols x nrows x npsfs x nints |
| 4 | ASDF | BINTABLE | N/A | variable

- SCI: 4-D data array containing a stack of 2-D PSF images aligned to each integration within a corresponding science target exposure. each integration.

- DQ: 4-D data array containing DQ flags for each PSF image.

- ERR: 4-D data array containing a stack of 2-D uncertainty estimates for each PSF image, per science target integration.

- ADSF: The data model meta data.



#### PSFSUB 

The klip step in the calwebb_coron3 pipeline subtracts an optimized combination of PSF images from each integration in a science target exposure. The resulting PSF-subtracted science exposure data uses the CubeModel data model, which when serialized to a FITS file has the structure shown below.

| HDU | EXTNAME | HDU Type | Data Type | Dimensions | 
| --- | --- | --- | --- | --- | 
| 0 | N/A | primary | N/A | N/A
| 1 | SCI | IMAGE | float32 | ncols x nrows x npsfs x nints | 
| 2 | DQ  | IMAGE | uint32 | ncols x nrows x npsfs xnints | 
| 3 | ERR | IMAGE | float32 | ncols x nrows x npsfs x nints |
| 4 | INT_TIMES | BINTABLE | N/A | nints (rows) x 7 cols
| 5 | VAR_POISSON | IMAGE | float32 | ncols x nrows x nints
| 6 | VAR_RNOISE | IMAGE | float32 | ncols x nrows x nints
| 7 | ASDF | BINTABLE | N/A | variable

- SCI: 3-D data array containing a stack of 2-D PSF-subtracted science target images, one per integration.

- ERR: 3-D data array containing a stack of 2-D uncertainty estimates for each science target integration.

- DQ: 3-D data array containing DQ flags for each science target integration.

- INT_TIMES: A table of beginning, middle, and end time stamps for each integration in the exposure.

- VAR_POISSON: 3-D data array containing the per-integration variance estimates for each pixel, based on Poisson noise only.

- VAR_RNOISE: 3-D data array containing the per-integration variance estimates for each pixel, based on read noise only.

- ADSF: The data model meta data.



# Proposal ID 1386

In [13]:
PROPOSAL_ID = '1194'
INSTRUMENT = 'NIRCAM'

In [14]:
#directory_1386_nircam = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/{PROPOSAL_ID}/mastDownload/JWST/'
directory_1386_nircam = f'/data/scratch/bariskurtkaya/dataset/{INSTRUMENT}/{PROPOSAL_ID}/mastDownload/JWST/'

## NIRCAM

---

In [19]:
#rateints_nircam = get_lower_products(suffix='rateints',directory=directory_1386_nircam)
calints_nircam = get_lower_products(suffix='calints',directory=directory_1386_nircam)

In [20]:
calints_nircam

[]

In [ ]:
#print('RATEINTS:',len(rateints_nircam))
print('CALINTS:',len(calints_nircam))

In [ ]:
calints_samples = calints_nircam[:15]

In [ ]:
header_calints,sci_calints,err_calints,dq_calints,_,_ = get_hdu(calints_samples,data='psfsub')

In [ ]:
plot_calints(calints_nircam,sci_calints,header_calints,title='CALINTS',instrume='NIRCAM')

---

In [ ]:
psfaligns_nircam = get_stage3_products(suffix='psfalign',directory=directory_1386_nircam)
psfsubs_nircam = get_stage3_products(suffix='psfsub',directory=directory_1386_nircam)
psfstacks_nircam = get_stage3_products(suffix='psfstack',directory=directory_1386_nircam)
crfints_nircam = get_stage3_products(suffix='crfints',directory=directory_1386_nircam)
i2ds_nircam = get_stage3_products(suffix='i2d',directory=directory_1386_nircam)

In [ ]:
print('PSFSTACK:',len(psfstacks_nircam))
print('I2D:',len(i2ds_nircam))
print('CRFINT:',len(crfints_nircam))
print('PSFSUB:',len(psfsubs_nircam))
print('PSFALIGN:',len(psfaligns_nircam))

In [ ]:
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')

In [ ]:
filtrs = get_headers(header_psfstacks,'FILTER')

---

## PSFSTACKS_1386_NIRCAM

### PSFSTACKS_SCI

In [ ]:
world_coords, axis_points = pixel2wcs(psfstacks_nircam[0],ispsf=True)
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')
instrume = get_headers(header_psfstacks,'INSTRUME')
targprop = get_headers(header_psfstacks,'TARGPROP')
program = get_headers(header_psfstacks,'PROGRAM')

plot_psfstack(psfstack=sci_psfstacks,
              ncol=9,
              nrow=2,
              title='PSFSTACK',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume=instrume,
              program=program,
              targprop=targprop
              )

In [ ]:
plot_psfstack(psfstack=err_psfstacks,
              ncol=9,
              nrow=2,
              title='PSFSTACK',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume=instrume,
              program=program,
              targprop=targprop)

### PSFSTACKS_DQ

In [ ]:
plot_psfstack(psfstack=dq_psfstacks,
              ncol=9,
              nrow=2,
              title='PSFSTACK',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume=instrume,
              program=program,
              targprop=targprop)

---

## I2D_1386_NIRCAM 

### I2D_SCI

In [ ]:
world_coords, axis_points = pixel2wcs(i2ds_nircam[0])
header_i2d,sci_i2d,err_i2d,dq_i2d,con_i2d,wht_i2d = get_hdu(i2ds_nircam,data='i2d')
filtrs = get_headers(header_i2d,'FILTER')
instrume = get_headers(header_i2d,'INSTRUME')
targprop = get_headers(header_i2d,'TARGPROP')
program = get_headers(header_i2d,'PROGRAM')
plot_i2d(data=sci_i2d,
             title='I2D',
             w=world_coords,
             axis_points=axis_points,
             filtrs=filtrs,
             instrume = instrume,
             program = program,
             targprop = targprop,
             ncols=5,
             save=True)

### I2D_ERR

In [ ]:
plot_i2d(data=err_i2d,
             title='I2D',
             w=world_coords,
             axis_points=axis_points,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop,
             ncols=5)

### I2D_CON

In [ ]:
squeezed_con_i2d = [np.squeeze(data) for data in con_i2d]
plot_i2d(data=squeezed_con_i2d,
             title='I2D',
             w=world_coords,
             axis_points=axis_points,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop,
             ncols=5)

### I2D_WHT

In [ ]:
plot_i2d(data=wht_i2d,
             title='I2D',
             w=world_coords,
             axis_points=axis_points,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop,
             ncols=5)

---

## PSFALIGN_1386_NIRCAM 

### PSFALIGN_SCI

In [ ]:
world_coords, axis_points = pixel2wcs(psfaligns_nircam[0],ispsf=True)
header_psfalign,sci_psfalign,err_psfalign,dq_psfalign,_,_ = get_hdu(psfaligns_nircam,data='psf')
filtrs = get_headers(header_psfalign,'FILTER')
instrume = get_headers(header_psfalign,'INSTRUME')
targprop = get_headers(header_psfalign,'TARGPROP')
program = get_headers(header_psfalign,'PROGRAM')
psfaligns_nircam(psfaligns=sci_psfalign,
              title='PSFALIGN',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop)

### PSFALIGN_ERR

In [ ]:
plot_psfaligns(psfaligns=err_psfalign,
              title='PSFALIGN',
              w=world_coords,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop,
              axis_points=axis_points)

### PSFALIGN_DQ

In [ ]:
plot_psfaligns(psfaligns=dq_psfalign,
              title='PSFALIGN',
              w=world_coords,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop,
              axis_points=axis_points)

---

## PSFSUB_1386_NIRCAM 

### PSFSUB_SCI

In [ ]:
world_coords, axis_points = pixel2wcs(psfsubs_nircam[0],ispsf=True)
header_psfsub,sci_psfsub,err_psfsub,dq_psfsub,_,_ = get_hdu(psfsubs_nircam,data='psfsub')
filtrs = get_headers(header_psfsub,'FILTER')
instrume = get_headers(header_psfsub,'INSTRUME')
targprop = get_headers(header_psfsub,'TARGPROP')
program = get_headers(header_psfsub,'PROGRAM')
plot_psfsubs(psfsubs=sci_psfsub,
              title='PSFSUB',
              w=world_coords,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop,
              axis_points=axis_points)

### PSFSUB_ERR

In [ ]:
plot_psfsubs(psfsubs=err_psfsub,
              title='PSFSUB',
              w=world_coords,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop,
              axis_points=axis_points)

### PSFSUB_DQ

In [ ]:
plot_psfsubs(psfsubs=dq_psfsub,
              title='PSFSUB',
              w=world_coords,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop,
              axis_points=axis_points)

---

## MIRI

In [ ]:
PROPOSAL_ID = '1386'

In [ ]:
directory_1386_miri = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/MIRI/{PROPOSAL_ID}/mastDownload/JWST/'

----

In [ ]:
rateints = get_lower_products(suffix='rateints',directory=directory_1386_miri)
calints_miri = get_lower_products(suffix='calints',directory=directory_1386_miri)

In [ ]:
print('RATEINTS:',len(rateints))
print('CALINTS:',len(calints_miri))

In [ ]:
idx_list = []
dims = check_hdu_dims(calints_miri,hdu=1)
for idx, shape, in enumerate(dims):
    if shape[0] <= 6:
        idx_list.append(idx)
        

In [ ]:
v_calints_miri = [calints_miri[i] for i in idx_list]

In [ ]:
header_calints_miri,sci_calints_miri,err_calints_miri,dq_calints_miri,_,_ = get_hdu(v_calints_miri,data='psf')
plot_calints(calints_miri,sci_calints_miri,header_calints_miri,title='CALINTS',instrume='MIRI')

In [ ]:
psfaligns_miri = get_stage3_products(suffix='psfalign',directory=directory_1386_miri)
psfsubs_miri = get_stage3_products(suffix='psfsub',directory=directory_1386_miri)
psfstacks_miri = get_stage3_products(suffix='psfstack',directory=directory_1386_miri)
crfints_miri = get_stage3_products(suffix='crfints',directory=directory_1386_miri)
i2ds_miri = get_stage3_products(suffix='i2d',directory=directory_1386_miri)

In [ ]:
print('PSFSTACK:',len(psfstacks_miri))
print('I2D:',len(i2ds_miri))
print('CRFINT:',len(crfints_miri))
print('PSFSUB:',len(psfsubs_miri))
print('PSFALIGN:',len(psfaligns_miri))

---

## PSFSTACKS_1386_MIRI

In [ ]:
world_coords, axis_points = pixel2wcs(psfstacks_miri[0],ispsf=True)
header_psfstack_miri,sci_psfstack_miri,err_psfstack_miri,dq_psfstack_miri,_,_ = get_hdu(psfstacks_miri,data='psf')
filtrs = get_headers(header_psfstack_miri,'FILTER')
instrume = get_headers(header_psfstack_miri,'INSTRUME')
targprop = get_headers(header_psfstack_miri,'TARGPROP')
program = get_headers(header_psfstack_miri,'PROGRAM')
plot_psfstack(psfstack= sci_psfstack_miri,
            ncol=9,
            nrow=2,
            title='PSFSTACK',
            w=world_coords,
            axis_points=axis_points,
            filtrs=filtrs,
            instrume = instrume,
            program = program,
            targprop = targprop)

## I2D_1386_MIRI

In [ ]:
world_coords, axis_points = pixel2wcs(i2ds_miri[0])
header_i2d_miri,sci_i2d_miri,err_i2d_miri,dq_i2d_miri,con_i2d,wht_i2d = get_hdu(i2ds_miri,data='i2d')
filtrs = get_headers(header_i2d_miri,'FILTER')
instrume = get_headers(header_i2d_miri,'INSTRUME')
targprop = get_headers(header_i2d_miri,'TARGPROP')
program = get_headers(header_i2d_miri,'PROGRAM')
plot_i2d_mir(data=sci_i2d_miri,
             title='I2D',
             w=world_coords,
             axis_points=axis_points,
             filtrs=filtrs,
             instrume = instrume,
             program = program,
             targprop = targprop,
             ncols=5,
             save=True)

In [ ]:
plot_i2d_mir(data=wht_i2d,
             title='I2D',
             w=world_coords,
             axis_points=axis_points,
             filtrs=filtrs,
             instrume = instrume,
             program = program,
             targprop = targprop,
             ncols=5)

In [ ]:
check_hdu_dims(i2ds_miri,hdu=3)

# Proposal ID 1441

In [ ]:
PROPOSAL_ID = '1441'

In [ ]:
directory_1441_nircam = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/{PROPOSAL_ID}/mastDownload/JWST/'

---

In [ ]:
rateints_nircam= get_lower_products(suffix='rateints',directory=directory_1441_nircam)
calints_nircam = get_lower_products(suffix='calints',directory=directory_1441_nircam)

In [ ]:
print('RATEINTS:',len(rateints_nircam))
print('CALINTS:',len(calints_nircam))

In [ ]:
psfaligns_nircam = get_stage3_products(suffix='psfalign',directory=directory_1441_nircam)
psfsubs_nircam = get_stage3_products(suffix='psfsub',directory=directory_1441_nircam)
psfstacks_nircam = get_stage3_products(suffix='psfstack',directory=directory_1441_nircam)
crfints_nircam = get_stage3_products(suffix='crfints',directory=directory_1441_nircam)
i2ds_nircam = get_stage3_products(suffix='i2d',directory=directory_1441_nircam)

In [ ]:
print('PSFSTACK:',len(psfstacks_nircam))
print('I2D:',len(i2ds_nircam))
print('CRFINT:',len(crfints_nircam))
print('PSFSUB:',len(psfsubs_nircam))
print('PSFALIGN:',len(psfaligns_nircam))

In [ ]:
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')

---

In [ ]:
world_coords, axis_points = pixel2wcs(psfstacks_nircam[0],ispsf=True)
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')
instrume = get_headers(header_psfstacks,'INSTRUME')
targprop = get_headers(header_psfstacks,'TARGPROP')
program = get_headers(header_psfstacks,'PROGRAM')
plot_psfstack(psfstack=sci_psfstacks,
              ncol=9,
              nrow=2,
              title='PSFSTACK',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop)

In [ ]:
sci_psfstacks[2].shape

In [ ]:
plot_psfstack(psfstack=err_psfstacks,
              ncol=9,
              nrow=2,
              title='PSFSTACK',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop)

In [ ]:
plot_psfstack(psfstack=dq_psfstacks,
              ncol=9,
              nrow=2,
              title='PSFSTACK',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume = instrume,
              program = program,
              targprop = targprop)

In [ ]:
world_coords, axis_points = pixel2wcs(i2ds_nircam[0])
header_i2d,sci_i2d,err_i2d,dq_i2d,con_i2d,wht_i2d = get_hdu(i2ds_nircam,data='i2d')
filtrs = get_headers(header_i2d,'FILTER')
instrume = get_headers(header_i2d,'INSTRUME')
targprop = get_headers(header_i2d,'TARGPROP')
program = get_headers(header_i2d,'PROGRAM')
plot_i2d(data=sci_i2d,
             title='I2D',
             w=world_coords,
             axis_points=axis_points,
             filtrs=filtrs,
             instrume = instrume,
             program = program,
             targprop = targprop,
             ncols=3)

In [ ]:
plot_i2d(data=err_i2d,
             title='I2D',
             w=world_coords,
             axis_points=axis_points,
             filtrs=filtrs,
             instrume = instrume,
             program = program,
             targprop = targprop,
             ncols=3)

# Proposal ID 2278

In [ ]:
PROPOSAL_ID = '2278'
directory_2278_nircam = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/{PROPOSAL_ID}/mastDownload/JWST/'

In [ ]:
psfaligns_nircam = get_stage3_products(suffix='psfalign',directory=directory_2278_nircam)
psfsubs_nircam = get_stage3_products(suffix='psfsub',directory=directory_2278_nircam)
psfstacks_nircam = get_stage3_products(suffix='psfstack',directory=directory_2278_nircam)
crfints_nircam = get_stage3_products(suffix='crfints',directory=directory_2278_nircam)
i2ds_nircam = get_stage3_products(suffix='i2d',directory=directory_2278_nircam)

In [ ]:
print('PSFSTACK:',len(psfstacks_nircam))
print('I2D:',len(i2ds_nircam))
print('CRFINT:',len(crfints_nircam))
print('PSFSUB:',len(psfsubs_nircam))
print('PSFALIGN:',len(psfaligns_nircam))

In [ ]:
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')

In [ ]:
world_coords, axis_points = pixel2wcs(psfstacks_nircam[0],ispsf=True)
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')
instrume = get_headers(header_psfstacks,'INSTRUME')
targprop = get_headers(header_psfstacks,'TARGPROP')
program = get_headers(header_psfstacks,'PROGRAM')

plot_psfstack(psfstack=sci_psfstacks,
              ncol=9,
              nrow=2,
              title='PSFSTACK',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume=instrume,
              program=program,
              targprop=targprop
              )

# Proposal ID 4454

In [ ]:
PROPOSAL_ID = '4454'
directory_4454_nircam = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/{PROPOSAL_ID}/mastDownload/JWST/'

In [ ]:
psfaligns_nircam = get_stage3_products(suffix='psfalign',directory=directory_4454_nircam)
psfsubs_nircam = get_stage3_products(suffix='psfsub',directory=directory_4454_nircam)
psfstacks_nircam = get_stage3_products(suffix='psfstack',directory=directory_4454_nircam)
crfints_nircam = get_stage3_products(suffix='crfints',directory=directory_4454_nircam)
i2ds_nircam = get_stage3_products(suffix='i2d',directory=directory_4454_nircam)

In [ ]:
print('PSFSTACK:',len(psfstacks_nircam))
print('I2D:',len(i2ds_nircam))
print('CRFINT:',len(crfints_nircam))
print('PSFSUB:',len(psfsubs_nircam))
print('PSFALIGN:',len(psfaligns_nircam))

In [ ]:
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')

In [ ]:
world_coords, axis_points = pixel2wcs(psfstacks_nircam[0],ispsf=True)
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')
instrume = get_headers(header_psfstacks,'INSTRUME')
targprop = get_headers(header_psfstacks,'TARGPROP')
program = get_headers(header_psfstacks,'PROGRAM')

plot_psfstack(psfstack=sci_psfstacks,
              ncol=9,
              nrow=2,
              title='PSFSTACK',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume=instrume,
              program=program,
              targprop=targprop
              )

# Proposal ID 1075

In [ ]:
PROPOSAL_ID = '1075'
directory_1075_nircam = f'/data/scratch/bariskurtkaya/dataset/NIRCAM/{PROPOSAL_ID}/mastDownload/JWST/'

In [ ]:
psfaligns_nircam = get_stage3_products(suffix='psfalign',directory=directory_1075_nircam)
psfsubs_nircam = get_stage3_products(suffix='psfsub',directory=directory_1075_nircam)
psfstacks_nircam = get_stage3_products(suffix='psfstack',directory=directory_1075_nircam)
crfints_nircam = get_stage3_products(suffix='crfints',directory=directory_1075_nircam)
i2ds_nircam = get_stage3_products(suffix='i2d',directory=directory_1075_nircam)

In [ ]:
print('PSFSTACK:',len(psfstacks_nircam))
print('I2D:',len(i2ds_nircam))
print('CRFINT:',len(crfints_nircam))
print('PSFSUB:',len(psfsubs_nircam))
print('PSFALIGN:',len(psfaligns_nircam))

In [ ]:
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')

In [ ]:
world_coords, axis_points = pixel2wcs(psfstacks_nircam[0],ispsf=True)
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')
instrume = get_headers(header_psfstacks,'INSTRUME')
targprop = get_headers(header_psfstacks,'TARGPROP')
program = get_headers(header_psfstacks,'PROGRAM')

plot_psfstack(psfstack=sci_psfstacks,
              ncol=9,
              nrow=2,
              title='PSFSTACK',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume=instrume,
              program=program,
              targprop=targprop
              )

# Proposal ID 1412

In [ ]:
PROPOSAL_ID = '1412'
directory_1412_nircam = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/{PROPOSAL_ID}/mastDownload/JWST/'

In [ ]:
psfaligns_nircam = get_stage3_products(suffix='psfalign',directory=directory_1412_nircam)
psfsubs_nircam = get_stage3_products(suffix='psfsub',directory=directory_1412_nircam)
psfstacks_nircam = get_stage3_products(suffix='psfstack',directory=directory_1412_nircam)
crfints_nircam = get_stage3_products(suffix='crfints',directory=directory_1412_nircam)
i2ds_nircam = get_stage3_products(suffix='i2d',directory=directory_1412_nircam)

In [ ]:
print('PSFSTACK:',len(psfstacks_nircam))
print('I2D:',len(i2ds_nircam))
print('CRFINT:',len(crfints_nircam))
print('PSFSUB:',len(psfsubs_nircam))
print('PSFALIGN:',len(psfaligns_nircam))

In [ ]:
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')

In [ ]:
world_coords, axis_points = pixel2wcs(psfstacks_nircam[0],ispsf=True)
header_psfstacks,sci_psfstacks,err_psfstacks,dq_psfstacks,_,_ = get_hdu(psfstacks_nircam,data='psf')
filtrs = get_headers(header_psfstacks,'FILTER')
instrume = get_headers(header_psfstacks,'INSTRUME')
targprop = get_headers(header_psfstacks,'TARGPROP')
program = get_headers(header_psfstacks,'PROGRAM')

plot_psfstack(psfstack=sci_psfstacks,
              ncol=9,
              nrow=2,
              title='PSFSTACK',
              w=world_coords,
              axis_points=axis_points,
              filtrs=filtrs,
              instrume=instrume,
              program=program,
              targprop=targprop
              )